In [2]:
!pip install opencv-python tensorflow numpy


Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# === File paths ===
video_path = "Screen Recording 2025-06-16 231138.mp4"
target_image_path = "c7af1a8a-6fc6-4a80-8c26-df0f88e6e222.png"
frame_size = (64, 64)
frame_skip = 5

# === Load and preprocess image ===
img = cv2.imread(target_image_path)
img = cv2.resize(img, frame_size) / 255.0
X_train = np.array([img])
y_train = np.array([1])

# === Build model ===
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy')
model.fit(X_train, y_train, epochs=10, verbose=1)

# === Scan video ===
cap = cv2.VideoCapture(video_path)
matches = []
index = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if index % frame_skip == 0:
        f = cv2.resize(frame, frame_size) / 255.0
        p = model.predict(np.array([f]), verbose=0)
        if p[0][0] > 0.9:
            t = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
            matches.append((index, t))
    index += 1

cap.release()

for idx, ts in matches:
    print(f"Match at frame {idx}, time {ts:.2f}s")
